In [4]:
! pip install julia

In [6]:
import julia

In [7]:
julia.install()


Precompiling PyCall...
Precompiling PyCall... DONE
PyCall is installed and built successfully.


In [1]:
from julia.Base import banner; banner()

In [2]:
! git clone https://github.com/gher-ulg/DIVAnd.py

Cloning into 'DIVAnd.py'...


In [5]:
! set PYTHONPATH="$HOME/src/DIVAnd.py/DIVAnd:$PYTHONPATH" python DIVAnd_argo.py

In [7]:
from julia import DIVAnd

In [8]:
help(DIVAnd.DIVAnd_cv)

Help on jlwrap object:

DIVAnd_cv = class jlwrap(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  jl_value
 |      Julia jl_value_t* (Any object)



In [14]:
import pandas as pd
import numpy as np
import os
from xarray import open_dataset
import time
from DIVAnd_cv import DIVAnd_cv

In [10]:
# -----------------------Choose data file and params-----------------------------
var_name = 'Oxy'
# standard_depth = 10
# year = 2010
# szn = 'OND'
year = 1991
szn = 'JFM'
# subsamp_interval_list = [50, 40, 30, 20, 10, 5]  # 3 or 5 is minimum possible

subsamp_interval_list = [10]

obs_dir = 'C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\value_vs_depth\\' \
          '14_sep_by_sl_and_year\\'

# GEBCO 6 minute mask
mask_dir = 'C:\\Users\\HourstonH\\Documents\\NEP_climatology\\data\\value_vs_depth\\' \
           '16_diva_analysis\\masks\\'

output_dir = 'C:\\Users\\HourstonH\\Documents\\NEP_climatology\\diva_explore\\' \
             'cross_validation\\'


In [15]:
# Iterate through file list
# for f in files:
for standard_depth in np.arange(0, 105, 5):
    # standard_depth = f[0]
    # year = f[1]
    # szn = f[2]
    # standard_depth = os.path.basename(f)[4:8]
    # year = os.path.basename(f)[10:14]
    # szn = os.path.basename(f)[15:18]

    # Initialize object to store new parameter estimates in
    corlen_list = []
    epsilon2_list = []

    # Read in standard level data file
    obs_filename = os.path.join(obs_dir + '{}_{}m_{}_{}.csv'.format(
        var_name, standard_depth, year, szn))

    print(os.path.basename(obs_filename))

    sldata = pd.read_csv(obs_filename)

    if sldata.empty:
        print('Warning: Dataframe empty')
        continue

    # Convert sldata's contents into numpy arrays
    xobs = np.array(sldata.Longitude)
    yobs = np.array(sldata.Latitude)
    vobs = np.array(sldata.SL_value)

    # print(min(xobs), max(xobs))
    # print(min(yobs), max(yobs))

    # Get boolean mask
    mask_filename = os.path.join(mask_dir + '{}_{}m_{}_{}_mask_6min.nc'.format(
        var_name, standard_depth, year, szn))

    mask_data = open_dataset(mask_filename)

    # Subset mask to speed up computations and avoid Cholesky factorization failure??
    mask_lon_subsetter = np.where(
        (mask_data.lon.data >= np.min(xobs) - 2) &
        (mask_data.lon.data <= np.max(xobs) + 2))[0]
    mask_lat_subsetter = np.where(
        (mask_data.lat.data >= np.min(yobs) - 2) &
        (mask_data.lat.data <= np.max(yobs) + 2))[0]

    for subsamp_interval in subsamp_interval_list:
        print()
        print('Subsampling interval:', subsamp_interval)

        # Reduce the subsetter for testing Cholesky factorization
        mask_lon_subsetter2 = mask_lon_subsetter[::subsamp_interval]
        mask_lat_subsetter2 = mask_lat_subsetter[::subsamp_interval]

        mask = mask_data.mask.data[mask_lat_subsetter2][:, mask_lon_subsetter2]
        print(mask.shape)
        Lon, Lat = np.meshgrid(mask_data.lon.data[mask_lon_subsetter2],
                               mask_data.lat.data[mask_lat_subsetter2])  # Create 2d arrays

        # ----------------------------Calculate input parameters----------------------------

        # Scale factor of the grid
        pm, pn = DIVAnd.metric(Lon, Lat)
        # print(pm, pn, sep='\n')

        # # For first guess correlation length, can use a value between 1/10 domain size and
        # # the domain size
        # # Also can try optimization on correlation length
        # domain_size_deg = -115-(-160)
        # deg2m = 111e3  # This is an approximation
        # domain_size_m = domain_size_deg * deg2m
        # print(domain_size_m/10)

        # Decreasing the correlation length decreases the "smoothness"
        lenx = 500e3  # 2.56  # 500e3  # 800e3  # in meters
        leny = 500e3  # 2.56  # 500e3  # 800e3  # in meters

        # error variance of the observations (normalized by the error variance of
        # the background field)
        # If epsilon2 is a scalar, it is thus the inverse of the signal-to-noise ratio
        signal_to_noise_ratio = 50.  # Default from Lu ODV session
        epsilon2 = 1/signal_to_noise_ratio  # 1.

        # Compute anomalies (i.e., subtract mean)
        vmean = np.mean(vobs)
        vanom = vobs - vmean

        # Choose number of testing points around the current value of L (corlen)
        nl = 1

        # Choose number of testing points around the current value of epsilon2
        ne = 1

        # Choose cross-validation method
        # 1: full CV; 2: sampled CV; 3: GCV; 0: automatic choice between the three
        method = 3

        # ------------------------Run cross-validation---------------------------------

        # Time the execution
        start_time = time.time()
        print('Start time: {}s'.format(start_time))

        bestfactorl, bestfactore, cvval, cvvalues, x2Ddata, y2Ddata, cvinter, xi2D, yi2D = DIVAnd_cv(
            mask, (pm, pn), (Lon, Lat), (xobs, yobs), vanom, (lenx, leny), epsilon2, nl, ne, method)

        execution_time = time.time() - start_time
        print('Execution time: {} sec'.format(execution_time))

        print('Bestfactorl:', bestfactorl)
        print('bestfactore:', bestfactore)
        # print('cvval:', cvval)
        # print('cvvalues:', cvvalues)
        # print('x2Ddata:', x2Ddata)
        # print('y2dData:', y2Ddata)
        # print('cvinter:', cvinter)
        # print('xi2D:', xi2D)
        # print('yi2D:', yi2D)

        print('New corlen:', lenx * bestfactorl)
        print('New epsilon2:', epsilon2 * bestfactore)

        corlen_list.append(lenx * bestfactorl)
        epsilon2_list.append(epsilon2 * bestfactore)

    # Make lists into a dataframe?
    # param_df = pd.DataFrame(
    #     data=np.array([subsamp_interval_list, corlen_list, epsilon2_list]).transpose(),
    #     columns=['interval_size', 'lenx', 'epsilon2'])
    param_df = pd.DataFrame(
        data=np.array([np.arange(0, 105, 5), corlen_list, epsilon2_list]).transpose(),
        columns=['standard_depth', 'lenx', 'epsilon2'])

    # param_df_filename = os.path.join(output_dir + 'cv_{}_{}m_{}_{}_nle_{}.csv'.format(
    #     var_name, standard_depth, year, szn, nl))
    param_df_filename = os.path.join(output_dir + 'cv_{}_to_100m_{}_{}_interval_{}.csv'.format(
        var_name, standard_depth, year, szn, subsamp_interval_list[0]))

    param_df.to_csv(param_df_filename, index=False)

Oxy_0m_1991_JFM.csv


ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
http://xarray.pydata.org/en/stable/user-guide/io.html 
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html

In [21]:
! conda install -c conda-forge xarray dask netCDF4 bottleneck

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed



Examining nbformat: 100%|##########| 340/340 [32:25<00:00,  2.75s/it]
                                                                     

Determining conflicts:   0%|          | 0/340 [00:00<?, ?it/s]
Examining conflict for scikit-learn anaconda nltk:   0%|          | 0/340 [00:00<?, ?it/s]
Examining conflict for scikit-learn dask networkx blas pywavelets mkl_random astropy numexpr pyerfa bkcharts patsy mkl-service numba imagecodecs scipy numpy pandas nltk numpy-base seaborn imageio xarray statsmodels mkl_fft scikit-image tifffile bottleneck matplotlib-base h5py matplotlib pytables bokeh:   0%|          | 1/340 [00:00<01:11,  4.76it/s]
Examining conflict for scikit-learn dask networkx blas pywavelets mkl_random astropy numexpr pyerfa bkcharts patsy mkl-service numba imagecodecs scipy numpy pandas nltk numpy-base seaborn imageio xarray statsmodels mkl_fft scikit-image tifffile bottleneck matplotlib-base h5py matplotlib pytables bokeh:   1%|          | 2/340 [00:00<00:35,  9.53it/s]


backports -> python -> ca-certificates
testpath -> python -> ca-certificates
nltk -> python -> ca-certificates
mistune -> python[version='>=2.7,<2.8.0a0'] -> ca-certificates
packaging -> python[version='>=2.7'] -> ca-certificates
diff-match-patch -> python -> ca-certificates
yapf -> python[version='>=2.7'] -> ca-certificates
pathlib2 -> python[version='>=2.7,<2.8.0a0'] -> ca-certificates
pytables -> python[version='>=2.7,<2.8.0a0'] -> ca-certificates
h5py -> python[version='>=2.7,<2.8.0a0'] -> ca-certificates
async_generator -> python[version='>2.7'] -> ca-certificates
pytz -> python -> ca-certificates
six -> python -> ca-certificates
rtree -> python[version='>=2.7,<2.8.0a0'] -> ca-certificates
conda-env -> python=2.7 -> ca-certificates
urllib3 -> python[version='<4.0'] -> ca-certificates
qtawesome -> python -> ca-certificates
attrs -> python[version='>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*,!=3.4.*'] -> ca-certificates
pyzmq -> python[version='>=2.7,<2.8.0a0'] -> ca-certificates
bottlene

xlwt -> python -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
pillow -> vc[version='14.*|9.*|>=14,<15.0a0|>=14.1,<15.0a0']
cython -> python[version='>=2.7,<2.8.0a0'] -> vc[version='10.*|>=9,<10.0a0']
openssl -> vc[version='10.*|14.*|9.*|>=14,<15.0a0|>=14.1,<15.0a0']
imagesize -> python[version='>=3.4'] -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
bitarray -> vc[version='14.*|9.*|>=14.1,<15.0a0|>=14,<15.0a0']
sqlite -> zlib[version='>=1.2.11,<1.3.0a0'] -> vc[version='>=9,<10.0a0']
anaconda==2021.05=py38_0 -> vc==14.2=h21ff451_1
pydocstyle -> python[version='>=3.6'] -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0|9.*']
sympy -> python[version='>=3.8,<3.9.0a0'] -> vc[version='10.*|14.*|>=14.1,<15.0a0|>=14,<15.0a0|9.*|>=9,<10.0a0']
numba -> python[version='>=2.7,<2.8.0a0'] -> vc[version='>=9,<10.0a0']
zeromq -> vc[version='14.*|14|9|9.*|>=14.1,<15.0a0|>=14,<15.0a0|>=9,<10.0a0']
heapdict -> python -> vc[version='10.*|14.*|>=

simplegeneric -> python -> pip
pycurl -> python[version='>=3.10,<3.11.0a0'] -> pip
python-dateutil -> python[version='>=3.6'] -> pip
sniffio -> python[version='>=3.9,<3.10.0a0'] -> pip
mkl-service -> python[version='>=3.9,<3.10.0a0'] -> pip
bkcharts -> python=3.6 -> pip
regex -> python[version='>=3.7,<3.8.0a0'] -> pip
toml -> python[version='>=2.7'] -> pip
async_generator -> python[version='>2.7'] -> pip
conda-content-trust -> python[version='>=3.5'] -> pip
backports.tempfile -> python -> pip
wincertstore -> python[version='>=3.7,<3.8.0a0'] -> pip
jupyterlab_pygments -> python -> pip
dask-core -> python[version='>=3.7'] -> pip
zict -> python -> pip
anaconda-client -> python[version='>=3.6'] -> pip
scikit-image -> python[version='>=3.7,<3.8.0a0'] -> pip
paramiko -> bcrypt[version='>=3.1.3'] -> pip
llvmlite -> python[version='>=3.7,<3.8.0a0'] -> pip
mock -> python[version='>=3.7,<3.8.0a0'] -> pip
joblib -> python[version='>=3.6'] -> pip
win_unicode_console -> python[version='>=3.8,<3.9.0

...

spyder -> chardet[version='>=2.0|>=2.0.0']
conda -> requests[version='>=2.18.4,<3'] -> chardet[version='>=3.0.2,<3.1.0|>=3.0.2,<4|>=3.0.2,<5']
anaconda-navigator -> requests -> chardet[version='>=3.0.2,<3.1.0|>=3.0.2,<4|>=3.0.2,<5']
anaconda-navigator -> chardet
anaconda==2021.05=py38_0 -> chardet==4.0.0=py38haa95532_1003
bokeh -> requests[version='>=1.2.3'] -> chardet[version='>=3.0.2,<3.1.0|>=3.0.2,<4|>=3.0.2,<5']

Package more-itertools conflicts for:
anaconda==2021.05=py38_0 -> pytest==6.2.3=py38haa95532_2 -> more-itertools[version='>=4.0.0']
importlib-metadata -> zipp[version='>=0.5'] -> more-itertools
importlib_metadata -> zipp[version='>=0.5'] -> more-itertools
zipp -> more-itertools
zipp -> jaraco.itertools -> more-itertools[version='>=4.0.0']
nltk -> pytest -> more-itertools[version='>=4.0,<6.0|>=4.0|>=4.0.0']
pytest -> more-itertools[version='>=4.0,<6.0|>=4.0|>=4.0.0']
astropy -> pytest[version='<4'] -> more-itertools[version='>=4.0,<6.0|>=4.0|>=4.0.0']
anaconda==2021.05=py38

spyder -> cloudpickle[version='>=0.5.0']
distributed -> cloudpickle[version='>=0.2.1|>=0.2.2|>=1.3.0|>=1.5.0']
scikit-image -> dask-core[version='>=0.15.0'] -> cloudpickle[version='>=0.2.2|>=1.1.1']
xarray -> dask[version='>=0.9'] -> cloudpickle[version='>=0.2.1|>=0.2.2']
spyder-kernels -> cloudpickle
anaconda==2021.05=py38_0 -> dask-core==2021.4.0=pyhd3eb1b0_0 -> cloudpickle[version='>=0.2.1|>=1.1.1|>=1.5.0|>=0.5.0']
scikit-image -> cloudpickle[version='>=0.2.1']
dask -> dask-core=2021.11.1 -> cloudpickle[version='>=1.1.1|>=1.5.0|>=1.3.0']

Package pysocks conflicts for:
requests -> urllib3[version='>=1.21.1,<1.27'] -> pysocks[version='>=1.5.6,<2.0,!=1.5.7']
anaconda==2021.05=py38_0 -> pysocks==1.7.1=py38haa95532_0
anaconda==2021.05=py38_0 -> urllib3==1.26.4=pyhd3eb1b0_0 -> pysocks[version='>=1.5.6,<2.0,!=1.5.7']
urllib3 -> pysocks[version='>=1.5.6,<2.0,!=1.5.7']

Package nest-asyncio conflicts for:
nbconvert -> nbclient[version='>=0.5.0,<0.6.0'] -> nest-asyncio[version='>=1.5']
widge

jupyterlab_server -> jupyter_server[version='>=1.4,<2'] -> traitlets[version='>=4.2.1|>=4.3']
nbclassic -> jupyter_server[version='>=1.8.0,<2.0.0'] -> traitlets[version='>=4.2.1|>=4.3']
jupyter_server -> jupyter_client[version='>=6.1.1'] -> traitlets[version='>=4.1.0|>=4.1.0,<6.0|>=4.1|>=4.2|>=5.0|>=5.1.0,<6.0']
ipython -> traitlets[version='>=4.2']
nbconvert -> nbformat[version='>=4.4'] -> traitlets[version='>=4.1']

Package openjpeg conflicts for:
matplotlib-base -> pillow[version='>=6.2.0'] -> openjpeg[version='>=2.3.0,<3.0a0|>=2.4.0,<2.5.0a0|>=2.4.0,<3.0a0']
bokeh -> pillow[version='>=7.1.0'] -> openjpeg[version='>=2.3.0,<3.0a0|>=2.4.0,<2.5.0a0|>=2.4.0,<3.0a0']
imageio -> pillow -> openjpeg[version='>=2.3.0,<3.0a0|>=2.4.0,<2.5.0a0|>=2.4.0,<3.0a0']
tifffile -> imagecodecs[version='>=2021.7.30'] -> openjpeg[version='>=2.3.0,<3.0a0|>=2.4.0,<2.5.0a0|>=2.3.1,<2.4.0a0']
anaconda==2021.05=py38_0 -> openjpeg==2.3.0=h5ec785f_1
imagecodecs -> openjpeg[version='>=2.3.0,<3.0a0|>=2.3.1,<2.4.0a0

jupyter_server -> jupyter-packaging[version='>=0.9,<1.0'] -> wheel
jupyterlab_server -> jupyter-packaging[version='>=0.9,<2'] -> wheel

Package typing_extensions conflicts for:
xarray -> typing_extensions[version='>=3.7']
xarray -> typing-extensions -> typing_extensions[version='3.10.0.0|3.10.0.0|3.10.0.1|3.10.0.2|4.0.0|3.7.4.3|3.7.4.2|3.7.4.1|3.7.4.1|3.7.4.1|3.7.4.1|3.7.4|3.7.2|3.7.2|3.6.6|3.6.6|3.6.5|3.10.0.2|3.7.4.3|3.7.4.1|3.7.4',build='py36_0|pyh06a4308_0|py27_0|py37_1000|py37_0|py36_0|py36h9f0ad1d_1|py36h9f0ad1d_2|py_0|py_0|pyha770c72_0|py38h32f6830_3|py27_1001|py27_1000|py27_0|pyh06a4308_0|pyh06a4308_0|py38_0']
astropy -> importlib-metadata -> typing_extensions[version='>=3.6.4']
distributed -> bokeh[version='>=0.12.3'] -> typing_extensions[version='>=3.10.0|>=3.7.4']
jsonschema -> importlib-metadata -> typing_extensions[version='>=3.6.4']
anaconda==2021.05=py38_0 -> black==19.10b0=py_0 -> typing_extensions[version='>=3.7.4']
black -> typing_extensions[version='>=3.10,!=3.10.0.1

anaconda==2021.05=py38_0 -> imagecodecs==2021.3.31=py38h5da4933_0 -> zfp[version='>=0.5.5,<1.0a0']
tifffile -> imagecodecs[version='>=2021.7.30'] -> zfp[version='>=0.5.5,<1.0a0']
anaconda==2021.05=py38_0 -> zfp==0.5.5=hd77b12b_6
imagecodecs -> zfp[version='>=0.5.5,<1.0a0']

Package platformdirs conflicts for:
spyder -> pylint[version='>=2.5.0'] -> platformdirs[version='>=2.2.0']
black -> platformdirs[version='>=2']
pylint -> platformdirs[version='>=2.2.0']
pyls-black -> black -> platformdirs[version='>=2']
python-language-server -> pylint[version='>=2.5.0'] -> platformdirs[version='>=2.2.0']

Package filelock conflicts for:
anaconda==2021.05=py38_0 -> filelock==3.0.12=pyhd3eb1b0_1
conda-build -> filelock

Package intervaltree conflicts for:
spyder -> intervaltree[version='>=3.0.2']
anaconda==2021.05=py38_0 -> spyder==4.2.5=py38haa95532_0 -> intervaltree[version='>=3.0.2']
anaconda==2021.05=py38_0 -> intervaltree==3.1.0=py_0

Package fastcache conflicts for:
sympy -> fastcache
anaconda=